In [1]:
# Stock Recommendations Aggregator
# Author: Balie
# Email: baliepages@gmail.com

In [2]:
import datetime as dt

#Cosmetic function to write comment to log
def drawLine(comment):
    try:
        print('', end = '\n')
        print(comment + ' ' * 10 + '-' * 10 + str(dt.datetime.now().strftime("%H:%M:%S")), end = '')
        print('\n')
    except Exception as e:
        errMsg = 'Error: Draw line'
        print(errMsg, repr(e))


In [3]:
import pandas as pd
pd.set_option('display.max_colwidth', 100)
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None 

iDf = pd.read_excel(r'StocksOutput\WatchList.xlsx', sheet_name='Stocks')
stockArray = iDf['Ticker'].tolist()
drawLine('Input read')


Input read          ----------15:45:32



In [4]:
import tldextract

loadStartTime = dt.datetime.now()
loadID = int(''.join(x for x in str(loadStartTime) if x.isdigit())[:14])

global urlArray, stockArray, siteArray, stockDf
urlArray = []
stockDf = pd.DataFrame(columns = ["siteName", "url", "stockCode", "snipCode", "snipText"])

global greedy, lazyloaders
greedy = ['marketbeat', 'zacks']
lazyLoaders = ['investorsobserver', 'weissratings', 'marketbeat', 'zacks']

global selectorDict
selectorDict = {
'wsj'           : 'snip = soup.select(".at8-col4 > .zonedModule")[0]',
'marketwatch'   : 'snip = soup.find(True, {"class" : "analyst-ratings"})',
'thestreet'     : 'snip = soup.select("#promo-div-quote-nav-unit-small-screen > div.quote-nav-rating-qr-label-container")[0]',
'zacks'         : ".rank_view",
'zacks2'        : "#premium_research > div > dl:nth-child(4) > dt > p",
'investorsobserver' : ".bar-label",
'weissratings'  : ".r-rating",
'gurufocus'     : 'snip = soup.select("#def_body_detail_height > font:nth-child(2)")',
'marketbeat'    : "#cphPrimaryContent_cphTabContent_pnlChart > div.row > div:nth-child(1) > div > div > div.col-sm-auto",
'marketbeat2'   : "#cphPrimaryContent_cphTabContent_pnlChart > div.row > div:nth-child(2) > div > div > div:nth-child(1) > h3",
'swingtradebot' : 'snip = soup.select("#main-content > div.row > div > article > div:nth-child(5) > div.card-body > div:nth-child(1) > table")',
'barchart'      : 'snip = soup.select("main#bc-main-content-wrapper div.background-widget.js-share-opinion-widget.less-margin")'
}

#Source websites
dummyStock = 'pangu'
wsjUrl     = 'https://www.wsj.com/market-data/quotes/' + dummyStock + '/research-ratings' 
mWatchUrl  = 'https://www.marketwatch.com/investing/stock/' + dummyStock + '/analystestimates'
strUrl     = 'https://realmoney.thestreet.com/quote/' + dummyStock + '.html'
zacUrl     = 'https://www.zacks.com/stock/quote/' + dummyStock 
invoUrl    = 'https://www.investorsobserver.com/symbols/' + dummyStock
wrUrl      = 'https://weissratings.com/stock/' + dummyStock + '-nasdaq'
gurUrl     = 'https://www.gurufocus.com/term/fscore/' + dummyStock + '/Piotroski-F-Score'
mBeatUrl   = 'https://www.marketbeat.com/stocks/NASDAQ/' + dummyStock + '/price-target'
swtUrl     = 'https://swingtradebot.com/equities/' + dummyStock 
barCUrl    = 'https://www.barchart.com/stocks/quotes/' + dummyStock 

siteArray  = [wsjUrl, mWatchUrl, strUrl, zacUrl, invoUrl, wrUrl, gurUrl, mBeatUrl, swtUrl, barCUrl]

#siteArray  = [swtUrl]
stockArray  = ['TGT', 'PDCE']
#urlArray.remove(['TWLO', 'https://www.wsj.com/market-data/quotes/TWLO/financials'])

stockArray  = sorted(stockArray)
siteArray  = sorted(siteArray)

#Build Site Url Array
for st in range(len(stockArray)):
    stockCode = stockArray[st]
    for si in range(len(siteArray)):
        site = siteArray[si]
        site = site.replace(dummyStock, stockCode)
        urlArray.append([stockCode, site])
        
drawLine('Url Array Created : {0} urls'.format(len(urlArray)))


Url Array Created : 20 urls          ----------15:45:33



In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import warnings
from cryptography.utils import CryptographyDeprecationWarning
warnings.filterwarnings("ignore", category=CryptographyDeprecationWarning)

drawLine('Selenium Function Defined')

import time
options = Options()
options.add_argument("--headless")
s=Service(r'C:\chromedriver.exe')
driver = webdriver.Chrome(service=s, options=options)
    
def seleniumScrape(stockCode, domain, url, selector):
    try:
        try:
            driver.get(url)
            time.sleep(1)
            snip_html = driver.find_element(By.CSS_SELECTOR, selector).get_attribute('innerHTML')
            snip_text = driver.find_element(By.CSS_SELECTOR, selector).text.strip()
        except Exception as e:
            print('Selenium Error: ', str(repr(e))[:100])
            if 'NASDAQ' in url.upper():
                newUrl = url.replace('NASDAQ','NYSE').replace('nasdaq','nyse')
                print('Retry with NYSE', newUrl)
                driver.get(newUrl)
                time.sleep(1)
                snip_html = driver.find_element(By.CSS_SELECTOR, selector).get_attribute('innerHTML')
                snip_text = driver.find_element(By.CSS_SELECTOR, selector).text.strip()
            
        if (domain in greedy):
            selector2 = selectorDict.get((domain + str(2)))
            snip2_html = driver.find_element(By.CSS_SELECTOR, selector2).get_attribute('innerHTML')
            snip2_text = driver.find_element(By.CSS_SELECTOR, selector2).text.strip()
            snip_html = snip_html + 'serppu' + snip2_html
            snip_text = snip_text + 'serppu' + snip2_text
        
        newRow = {'siteName':domain, 'url':url, 'stockCode':stockCode, 'snipCode':str(snip_html), 'snipText': snip_text}
        return newRow

    except Exception as e:
        print('Final Selenium Error: ', str(repr(e))[:100])


Selenium Function Defined          ----------15:45:34



In [6]:
from bs4 import BeautifulSoup
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

scrapeStartTime = dt.datetime.now()
drawLine('Scraping Started... Grab your coffee! ')

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}

urlCount = len(urlArray)
for i in range(len(urlArray)):
    try:
        stockCode = urlArray[i][0]
        url = urlArray[i][1]
        parsedUrl = tldextract.extract(url)
        domain = parsedUrl.domain
        selector = selectorDict.get(domain)
        print(str(i) + '/' + str(urlCount) + ' : ', stockCode + ' : ', domain + ' : ', url)

        #Using Selenium for lazy loading sites
        if (domain in lazyLoaders):
            newRow = seleniumScrape(stockCode, domain, url, selector)
            stockDf = stockDf.append(newRow, ignore_index=True)
        else:
            try:
                r = requests.get(url, headers=headers, allow_redirects=False, verify=False)
                soup = BeautifulSoup(r.text)
                exec(selector)
            except Exception as e:
                print('BS4 Error: ', str(repr(e))[:100])
                if 'NASDAQ' in url.upper():
                    newUrl = url.replace('NASDAQ','NYSE').replace('nasdaq','nyse')
                    print('Retry with NYSE', newUrl)
                    r = requests.get(newUrl, headers=headers, allow_redirects=False, verify=False)
                    soup = BeautifulSoup(r.text)
                    exec(selector)

                
            try:
                snip_text = snip.get_text().strip()
            except Exception as e:
                snip_text = snip[0].get_text().strip()
            snip = str(snip)

            if (domain in greedy):
                selector = selectorDict.get((domain + str(2)))
                exec(selector)
                snip2_text = snip2.get_text().strip()
                snip2 = str(snip2)
                snip = snip + 'serppu' + snip2
                snip_text = snip_text + 'serppu' + snip2_text

            newRow = {'siteName':domain, 'url':url, 'stockCode':stockCode, 'snipCode':snip, 'snipText': snip_text}
            stockDf = stockDf.append(newRow, ignore_index=True)

    except Exception as e:
        print('Final BS4 Error: ', str(repr(e))[:100])
        
scrapeEndTime = dt.datetime.now()
loadDuration = str(scrapeEndTime - scrapeStartTime).split('.')[0]
drawLine('Scraping Completed in ' + loadDuration)


Scraping Started... Grab your coffee!           ----------15:45:37

0/20 :  PDCE :  thestreet :  https://realmoney.thestreet.com/quote/PDCE.html
1/20 :  PDCE :  swingtradebot :  https://swingtradebot.com/equities/PDCE
2/20 :  PDCE :  weissratings :  https://weissratings.com/stock/PDCE-nasdaq
3/20 :  PDCE :  barchart :  https://www.barchart.com/stocks/quotes/PDCE
4/20 :  PDCE :  gurufocus :  https://www.gurufocus.com/term/fscore/PDCE/Piotroski-F-Score
5/20 :  PDCE :  investorsobserver :  https://www.investorsobserver.com/symbols/PDCE
6/20 :  PDCE :  marketbeat :  https://www.marketbeat.com/stocks/NASDAQ/PDCE/price-target
7/20 :  PDCE :  marketwatch :  https://www.marketwatch.com/investing/stock/PDCE/analystestimates
8/20 :  PDCE :  wsj :  https://www.wsj.com/market-data/quotes/PDCE/research-ratings
9/20 :  PDCE :  zacks :  https://www.zacks.com/stock/quote/PDCE
10/20 :  TGT :  thestreet :  https://realmoney.thestreet.com/quote/TGT.html
11/20 :  TGT :  swingtradebot :  https://swingtrad

In [7]:
dflist = [stockDf]

for x in ['snipText', 'snipCode']:
    stockDf[x] = stockDf[x].apply(pd.to_numeric, errors = 'ignore')

stockDfFile = "StocksOutput/stockDf_" + str(loadID) + ".xlsx"
Excelwriter = pd.ExcelWriter(stockDfFile, engine="xlsxwriter")
for i, df in enumerate (dflist):
    df.to_excel(Excelwriter, sheet_name="Sheet" + str(i+1), index=False)
Excelwriter.save()
drawLine('Saved scraped data')


Saved scraped data          ----------15:46:27



In [8]:
#stockDfFile = "StocksOutput/stockDf_" + str(loadID) + ".xlsx"
stockDfFile = "StocksOutput/stockDf_20220417120821.xlsx"
stockDf = pd.read_excel(stockDfFile)
print(stockDf.shape)
drawLine('Read from persistent storage')

(1396, 5)

Read from persistent storage          ----------15:46:27



In [9]:
df = stockDf.copy()
df.sort_values(by=['siteName', 'stockCode'], inplace=True, ascending=True)

drawLine('Fill missing snipText with snipCode for investorsobserver')
df['snipText'] = df.snipText.combine_first(df.snipCode[:3])
df.query('siteName in ["investorsobserver"]')[:2]


Fill missing snipText with snipCode for investorsobserver          ----------15:46:27



,siteName,url,stockCode,snipCode,snipText
5,investorsobserver,https://www.investorsobserver.com/symbols/AAPL,AAPL,64,65
14,investorsobserver,https://www.investorsobserver.com/symbols/ABBV,ABBV,60,61


In [10]:
import json

drawLine('Manipulate tabular data')

def renderHtml(snip):
    try:
        return '{0}.prettify.stripped_strings'.format(snip)
    except Exception as e:
        pass

def getTableData(snip):
    try:
        return [[cell.text for cell in row("td")] for row in BeautifulSoup(snip)("tr")]
    except Exception as e:
        pass
    
def linearizeTable(snip):
    try:
        return [list(map(lambda x: str(x).replace('\n',''), item)) for item in snip if item]
    except Exception as e:
        pass
    
def tableDataToDict(data):
    try:
        cols = ['Rating', '3M', '1M', 'Current']
        df = pd.DataFrame(data, columns=cols)
        return df.set_index('Rating').T.to_dict('list')
    except Exception as e:
        pass
    
def listToDict(lister):
    try:
        return dict(lister)
    except Exception as e:
        pass

df['snipCode_v1'] = df['snipCode'].apply(getTableData)
df['snipCode_v2'] = df['snipCode_v1'].apply(linearizeTable)
df['snipCode_v3'] = df['snipCode_v2'].apply(tableDataToDict)
df['snipCode_v2'] = df['snipCode_v2'].apply(listToDict)
#df.style.format({'SnipCode': renderHtml})
df[:2]


Manipulate tabular data          ----------15:46:27



,siteName,url,stockCode,snipCode,snipText,snipCode_v1,snipCode_v2,snipCode_v3
3,barchart,https://www.barchart.com/stocks/quotes/AAPL,AAPL,"[<div class=""background-widget js-share-opinion-widget less-margin""> <h3>Barchart Technical Opin...",Barchart Technical Opinion buy The Barchart Technical Opinion rating is a 40% Buy with ...,[],{},{}
12,barchart,https://www.barchart.com/stocks/quotes/ABBV,ABBV,"[<div class=""background-widget js-share-opinion-widget less-margin""> <h3>Barchart Technical Opin...",Barchart Technical Opinion Strong buy The Barchart Technical Opinion rating is a 80% ...,[],{},{}


In [11]:
#Recommendation Matrix
drawLine('Build Recommendations')
df['snipText'] = df['snipText'].apply(lambda x: str(x).replace('\n', '')
                                                 .replace('TheStreetQuant Rating:', '')
                                                 .replace('(As of Today)', '')
                                                 .replace(':', '')
                                                 .replace('Based on','')
                                                 .replace('Analyst Ratings','')
                                                 .replace('serppu',''
                                                .replace('()', ''))
                                     )
recDf = (df.query('siteName in ["zacks", "thestreet", "investorsobserver", "weissratings", "gurufocus", "marketbeat", "barchart"]')
          #.query('snipCode.str.contains("Buy")')
        #[['stockCode', 'siteName', 'url', 'snipText']]
)
recDfLinear = pd.pivot_table(recDf, values = 'snipText', index=['stockCode'], columns = 'siteName', aggfunc='first').reset_index()
recDfLinear['thestreet'] = recDfLinear['thestreet'].apply(lambda x:'C (Hold)' if str(x).strip() == '()' else x )
recDfLinear['thestreet'] = recDfLinear['thestreet'].fillna(0)
recDfLinear['gurufocus'] = recDfLinear['gurufocus'].astype(float)
recDfLinear[:2]


Build Recommendations          ----------15:46:39



siteName,stockCode,barchart,gurufocus,investorsobserver,marketbeat,thestreet,weissratings,zacks
0,AAPL,Barchart Technical Opinion buy The Barchart Technical Opinion rating is a 40% Buy with ...,7.0,65,BUY 31 $191.2415.70% Upside,A (Buy),A,2-Buyof 5 2D Value | A Growth | A Momentum | B VGM
1,ABBV,Barchart Technical Opinion Strong buy The Barchart Technical Opinion rating is a 80% ...,7.0,61,BUY 16 $156.40-3.64% Downside,A+ (Buy),B,3-Holdof 5 3B Value | B Growth | A Momentum | A VGM


In [12]:

#Flatten data structures in table data
drawLine('Flatten data structures in table data')

json_struct = json.loads(df.to_json(orient="records"))    
dfFlat = pd.json_normalize(json_struct)
dfFlat.rename(columns=lambda x: x.replace('snipCode_v3.','').replace('snipCode_v2.',''), inplace=True)

dfFlat[:2]


Flatten data structures in table data          ----------15:46:39



,siteName,url,stockCode,snipCode,snipText,snipCode_v1,snipCode_v2,Buy,Overweight,Hold,Underweight,Sell,Consensus,Weak or Absent,Strong,Very Strong,snipCode_v3,High,Median,Low,Average,Current Price
0,barchart,https://www.barchart.com/stocks/quotes/AAPL,AAPL,"[<div class=""background-widget js-share-opinion-widget less-margin""> <h3>Barchart Technical Opin...",Barchart Technical Opinion buy The Barchart Technical Opinion rating is a 40% Buy with ...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,barchart,https://www.barchart.com/stocks/quotes/ABBV,ABBV,"[<div class=""background-widget js-share-opinion-widget less-margin""> <h3>Barchart Technical Opin...",Barchart Technical Opinion Strong buy The Barchart Technical Opinion rating is a 80% ...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Short, Mid and Long Term Trends
drawLine('Capture Short, Mid and Long Term Trends')
if 'Weak or Absent' in dfFlat.columns.values:
    dfFlat.columns = dfFlat.columns.str.replace('Weak or Absent', 'Long Mid Short')

dfForecast = dfFlat.query('siteName == "swingtradebot"')
dfForecast = dfForecast.fillna('['', '', '']')

dfForecast['Long'] = dfForecast['Long Mid Short'].apply(lambda x: x[0].replace('[', '').replace(',', ''))
dfForecast['Mid'] = dfForecast['Long Mid Short'].apply(lambda x: x[1].replace('[', '').replace(',', ''))
dfForecast['Short'] = dfForecast['Long Mid Short'].apply(lambda x: x[2].replace('[', '').replace(',', ''))
    
dfForecast = dfForecast[['stockCode', 'Long', 'Mid', 'Short']]
dfForecast[:2]


Capture Short, Mid and Long Term Trends          ----------15:46:39



,stockCode,Long,Mid,Short
721,AAPL,Up,Down,Down
722,ABBV,,,


In [14]:
# Buy Sell Trends over 6 months
drawLine('Capture Buy Sell Trends over 6 months')

dfTrend = dfFlat.query('siteName == "marketwatch"')[['stockCode', 'Consensus']]
dfTrend[:2]


Capture Buy Sell Trends over 6 months          ----------15:46:39



,stockCode,Consensus
577,AAPL,"[Overweight, Overweight, Overweight]"
578,ABBV,"[Overweight, Overweight, Overweight]"


In [15]:
# Current and Average Price
drawLine('Current and Average Price')

dfPrice = dfFlat.query('siteName == "wsj"')[['stockCode', 'Average', 'Current Price']]
for x in ['Average', 'Current Price']:
    dfPrice[x] = dfPrice[x].apply(lambda x: x.replace(',', '')
                                             .replace('$', '')
                                             .replace('¥', '')
                                             .strip()
                                             ).astype(float)
dfPrice = dfPrice.fillna(0)
dfPrice[:2]


Current and Average Price          ----------15:46:39



,stockCode,Average,Current Price
1138,ABBV,164.10,162.31
1139,ABC,164.67,161.21


In [16]:
#Merge all dataframes
drawLine('Merged Dataframe')
df1 = recDfLinear.merge(dfTrend, how = 'outer', on='stockCode')
df2 = df1.merge(dfForecast, how = 'outer', on='stockCode')
dfMerge = df2.merge(dfPrice, how = 'outer', on='stockCode')
dfMerge['Current Price'] = dfMerge['Current Price'].fillna(0)
dfMerge['Average'] = dfMerge['Average'].fillna(0)
dfMerge[:2]


Merged Dataframe          ----------15:46:39



,stockCode,barchart,gurufocus,investorsobserver,marketbeat,thestreet,weissratings,zacks,Consensus,Long,Mid,Short,Average,Current Price
0,AAPL,Barchart Technical Opinion buy The Barchart Technical Opinion rating is a 40% Buy with ...,7.0,65,BUY 31 $191.2415.70% Upside,A (Buy),A,2-Buyof 5 2D Value | A Growth | A Momentum | B VGM,"[Overweight, Overweight, Overweight]",Up,Down,Down,0.0,0.00
1,ABBV,Barchart Technical Opinion Strong buy The Barchart Technical Opinion rating is a 80% ...,7.0,61,BUY 16 $156.40-3.64% Downside,A+ (Buy),B,3-Holdof 5 3B Value | B Growth | A Momentum | A VGM,"[Overweight, Overweight, Overweight]",,,,164.1,162.31


In [17]:
#VGM Score
drawLine('Introduce calculated fields')
try:
    dfMerge['zacks'] = dfMerge['zacks'].fillna('3-Holdof 5 C Value | C Growth | C Momentum | C VGM')
    dfMerge['zacks'] = dfMerge['zacks'].replace(to_replace='NA Value | NA Growth | NA Momentum | NA VGM', value='3-Holdof 5 C Value | C Growth | C Momentum | C VGM')
    dfMerge['zacksScore'] = dfMerge['zacks'].apply(lambda x: x.split('of 5')[0].split('-')[1].strip())
    dfMerge['VGM'] = dfMerge['zacks'].apply(lambda x: x.split('|')[-1].replace('VGM', '').strip())
    dfMerge['Value'] = dfMerge['zacks'].apply(lambda x: x.split('Value')[0].split(' ')[-2].replace('5','').strip()[-1])
    dfMerge['Growth'] = dfMerge['zacks'].apply(lambda x: x.split('Growth')[0].split(' ')[-2].strip())
    dfMerge['Momentum'] = dfMerge['zacks'].apply(lambda x: x.split('Momentum')[0].split(' ')[-2].strip())
except Exception as e:
    drawLine('Zacks ERROR : ' + repr(e))

#Downgrade/Upgrade
try:
    dfMerge['Consensus'] = dfMerge['Consensus'].fillna('000')
    dfMerge['mwPrevRec'] = dfMerge['Consensus'].apply(lambda x: x[-1])
    dfMerge['mwCurrRec'] = dfMerge['Consensus'].apply(lambda x: x[-2])
except Exception as e:
    drawLine('MarketWatch ERROR : ' + repr(e))

#MarketBeat
try:
    dfMerge['marketbeat'] = dfMerge['marketbeat'].fillna('Hold $0.0 0% Upside')
    dfMerge['marketbeat'] = dfMerge['marketbeat'].apply(lambda x: 'Hold $0.0 0% Upside' if '$' not in x else x)
    dfMerge['mbRec'] = dfMerge['marketbeat'].apply(lambda x: x.split('$')[0].split(' ')[0].capitalize())
    dfMerge['mbTgtPrice'] = dfMerge['marketbeat'].apply(lambda x: x.split('$')[1].split('.')[0])
except Exception as e:
    drawLine('MarketBeat ERROR : ' + repr(e))
    
#dfMerge.drop(['marketbeat', 'zacks', 'Consensus'], axis=1, inplace=True)

dollarColumns = ['Current Price', 'Average']
for x in dollarColumns:
    dfMerge[x] = dfMerge[x].apply(lambda x: str(x).replace('$', ''))

df = dfMerge.copy()
dfMerge[:2]


Introduce calculated fields          ----------15:46:39



,stockCode,barchart,gurufocus,investorsobserver,marketbeat,thestreet,weissratings,zacks,Consensus,Long,Mid,Short,Average,Current Price,zacksScore,VGM,Value,Growth,Momentum,mwPrevRec,mwCurrRec,mbRec,mbTgtPrice
0,AAPL,Barchart Technical Opinion buy The Barchart Technical Opinion rating is a 40% Buy with ...,7.0,65,BUY 31 $191.2415.70% Upside,A (Buy),A,2-Buyof 5 2D Value | A Growth | A Momentum | B VGM,"[Overweight, Overweight, Overweight]",Up,Down,Down,0.0,0.0,Buy,B,D,A,A,Overweight,Overweight,Buy,191
1,ABBV,Barchart Technical Opinion Strong buy The Barchart Technical Opinion rating is a 80% ...,7.0,61,BUY 16 $156.40-3.64% Downside,A+ (Buy),B,3-Holdof 5 3B Value | B Growth | A Momentum | A VGM,"[Overweight, Overweight, Overweight]",,,,164.1,162.31,Hold,A,B,B,A,Overweight,Overweight,Buy,156


In [18]:
#Sell Code to Numeric Score
drawLine('Sell Code to Numeric Score')
ratingDict = {
    'Strong Buy' : 3,
    'Buy' : 2,
    'Overweight' : 1,
    'Hold' : 0,
    'Underweight' : -1,
    'Sell' : -2,
    'Strong Sell' : -3,
    'A' : 3,
    'B' : 2,
    'C' : 1,
    'D' : 0,
    'E' : -1,
    'F' : -2,
    'A+ (Buy)' : 3,
    'A (Buy)' : 2.8,
    'A- (Buy)' : 2.6,
    'B+ (Buy)' : 2.4,
    'B (Buy)' : 2.2,
    'B- (Buy)' : 2,
    'C+ (Hold)' : 1,
    'C (Hold)' : 0,
    'C- (Hold)' : -1,
    'D+ (Sell)' : -2,
    'D (Sell)' : -2.2,
    'D- (Sell)' : -2.4,
    'E+ (Sell)' : -2.6,
    'E (Sell)' : -2.8,
    'E- (Sell)' : -3,
    'N/A' : 0
}

RecCodeCols = ['thestreet', 'weissratings', 'zacksScore', 'VGM', 'Value', 'Growth', 'Momentum', 'mwPrevRec', 'mwCurrRec', 'mbRec', 'mbTgtPrice']
for x in RecCodeCols:
    df.replace({x: ratingDict}, inplace=True)
    
try:
    df = df.fillna(0)
    dfNumeric = df.apply(pd.to_numeric, errors = 'ignore')
except Exception as e:
    print('Error in converting to number' + repr(e))

dfNumeric[:2]


Sell Code to Numeric Score          ----------15:46:40



,stockCode,barchart,gurufocus,investorsobserver,marketbeat,thestreet,weissratings,zacks,Consensus,Long,Mid,Short,Average,Current Price,zacksScore,VGM,Value,Growth,Momentum,mwPrevRec,mwCurrRec,mbRec,mbTgtPrice
0,AAPL,Barchart Technical Opinion buy The Barchart Technical Opinion rating is a 40% Buy with ...,7.0,65,BUY 31 $191.2415.70% Upside,2.8,3.0,2-Buyof 5 2D Value | A Growth | A Momentum | B VGM,"[Overweight, Overweight, Overweight]",Up,Down,Down,0.0,0.00,2,2,0,3,3,1,1,2,191
1,ABBV,Barchart Technical Opinion Strong buy The Barchart Technical Opinion rating is a 80% ...,7.0,61,BUY 16 $156.40-3.64% Downside,3.0,2.0,3-Holdof 5 3B Value | B Growth | A Momentum | A VGM,"[Overweight, Overweight, Overweight]",,,,164.1,162.31,0,3,2,2,3,1,1,2,156


In [19]:
#Calculated fields metrics
drawLine('Calculated fields metrics')

def setMaxUpside(x):
    if x>= 25:
        return 25
    elif x<=-25:
        return -25
    else:
        return x

dfNumeric['__AvgTgtPrice'] = dfNumeric[['Average', 'mbTgtPrice' ]].mean(axis=1).astype(int)
dfNumeric.loc[(dfNumeric[['Average', 'mbTgtPrice' ]] == 0).any(axis=1), '__AvgTgtPrice'] = dfNumeric[['Average', 'mbTgtPrice' ]].max(axis=1).astype(int)

dfNumeric['__Upside'] = ((dfNumeric['__AvgTgtPrice'] - dfNumeric['Current Price']) / dfNumeric['Current Price'])*100
dfNumeric['__Upside'] = dfNumeric['__Upside'].apply(setMaxUpside)

#Existence of Buy or Sell
recCols = ['thestreet', 'weissratings', 'zacksScore', 'VGM', 'mwCurrRec', 'mbRec']
dfNumeric['SellSignal'] = (dfNumeric[recCols] < -1 ).any(axis=1).astype(int) * -1
dfNumeric['BuySignal'] = (dfNumeric[recCols] > 2.2 ).any(axis=1).astype(int) * 1
dfNumeric['__StrongBuySellSignal'] = dfNumeric['SellSignal'] + dfNumeric['BuySignal']

dfNumeric['__Recommendation'] = dfNumeric[recCols].mean(axis=1)

#MarketWatch Downgraded Flag
dfNumeric['mwDowngrade'] = (df['mwCurrRec'] < df['mwPrevRec']).astype(int) * -1
dfNumeric['mwUpgrade'] = (df['mwCurrRec'] > df['mwPrevRec']).astype(int) * 1
dfNumeric['__mwChange'] = dfNumeric['mwUpgrade'] + dfNumeric['mwDowngrade']

__Cols = [col for col in dfNumeric.columns if '__' in col]
for x in ['stockCode', 'Average', 'investorsobserver', 'gurufocus']:
    __Cols.append(x)
dfNumeric = dfNumeric[__Cols]
dfNumeric[:2]


Calculated fields metrics          ----------15:46:40



,__AvgTgtPrice,__Upside,__StrongBuySellSignal,__Recommendation,__mwChange,stockCode,Average,investorsobserver,gurufocus
0,191,25.000000,1,2.133333,0,AAPL,0.0,65,7.0
1,160,-1.423203,1,1.833333,0,ABBV,164.1,61,7.0


In [20]:
#Create finalDf
#Introduce barchart recommendation separate from other buy recommendations
#barchart recommendation includes market dynamics in addition to technical details

drawLine('Create finalDf')

dfMerge.drop(['Average', 'investorsobserver', 'gurufocus'], axis=1, inplace=True)
finalDf = dfMerge.merge(dfNumeric, how = 'outer', on='stockCode')
finalDf[:2]

bcDefault = 'Barchart Technical Opinion N/A The Barchart Technical Opinion rating is a 100% Hold with a Weakest short term outlook on maintaining the current direction.'
dfMerge['barchart'] = dfMerge['barchart'].fillna(bcDefault)
finalDf['barCRec'] = dfMerge['barchart'].apply(lambda x: x.split('The Barchart Technical Opinion rating is a ')[0]
                                                .replace('Barchart Technical Opinion', '')
                                                .strip())
finalDf['barCScore'] = dfMerge['barchart'].apply(lambda x: x.split('The Barchart Technical Opinion rating is a ')[1]
                                                 .split('%')[0]
                                                .strip())
finalDf['barchart'] = dfMerge['barchart'].apply(lambda x: x.replace('The Barchart Technical Opinion rating is a ', '')
                                                .replace('Barchart Technical Opinion', '')
                                                .replace('Share', '')
                                                .replace('See More', '')
                                                .strip())


Create finalDf          ----------15:46:40



In [21]:
#Sort and select Columns for final report
drawLine('Sort and select Columns for final report')

selectCols = ['stockCode', 'Current Price', 'mbTgtPrice', 'Average', '__AvgTgtPrice', '__Upside', 
              'barCRec', 'barCScore', '__Recommendation', 
              '__StrongBuySellSignal',
              'investorsobserver', 'gurufocus',
              'Value', 'Growth', 'Momentum', 'VGM',
              'thestreet', 'weissratings', 'zacksScore', 'mwCurrRec', 'mbRec',
              'Short', 'Mid', 'Long'
             ]
finalDf = finalDf[selectCols]
finalDf.sort_values(by=['__Recommendation', '__Upside', 'Value', '__StrongBuySellSignal'], inplace=True, ascending=False)
finalDf = finalDf.apply(pd.to_numeric, errors = 'ignore')
finalDf[:2]


Sort and select Columns for final report          ----------15:46:40



,stockCode,Current Price,mbTgtPrice,Average,__AvgTgtPrice,__Upside,barCRec,barCScore,__Recommendation,__StrongBuySellSignal,investorsobserver,gurufocus,Value,Growth,Momentum,VGM,thestreet,weissratings,zacksScore,mwCurrRec,mbRec,Short,Mid,Long
0,AAPL,0.00,191,0.00,191,25.000000,buy,40,2.133333,1,65,7.0,D,A,A,B,A (Buy),A,Buy,Overweight,Buy,Down,Down,Up
97,PDCE,77.87,0,92.47,92,18.145627,Strong buy,100,2.033333,1,nan,9.0,C,B,B,A,B (Buy),B,Strong Buy,Buy,Hold,Up,Up,Up


In [22]:
#Write final results to excel
drawLine('Write final results to excel')

#List of all Dataframes to be written to the file
dflist = [stockDf, dfMerge, dfNumeric, finalDf]
#dflist = [stockDf]

tgtFileName = "StocksOutput/StockMarket_" + str(loadID) + ".xlsx"
Excelwriter = pd.ExcelWriter(tgtFileName, engine="xlsxwriter")
for i, df in enumerate (dflist):
    df.to_excel(Excelwriter, sheet_name="Sheet" + str(i+1), index=False)
Excelwriter.save()

loadEndTime = dt.datetime.now()
loadDuration = str(loadEndTime - loadStartTime).split('.')[0]
drawLine(str(loadID) + ' Load Completed in ' + loadDuration)


Write final results to excel          ----------15:46:40


20220417154533 Load Completed in 0:01:08          ----------15:46:42

